In [ ]:
# 
# Copyright (c) 2022 Go2Market Insights d/b/a Analyzr
# All rights reserved
# https://analyzr.ai
# 
# The above copyright notice and this permission notice shall be included in all copies or substantial portions
# of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO
# THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT,
# TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
# 
# For access to the original model go to https://app.analyzr.ai
# For support and documentation see comments below or go to https://support.analyzr.ai
# 
# To run this notebook you will need the following Python packages:
#   - pandas
#   - numpy
#   - analyzrclient
#   - matplotlib
#   - seaborn
#
# This notebook was automatically generated by Analyzr.
#
import pandas as pd
import numpy as np
import sys, os, uuid
from copy import deepcopy
import datetime 

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

In [ ]:
# 
# LOAD DATA
# 
# This notebook assumes your original source file is in the same folder as this Jupyter
# notebook. You may need to update the path below. 
# 
raw_data = pd.read_csv('https://g2mstaticfiles.blob.core.windows.net/$web/titanic.csv', encoding = "ISO-8859-1", low_memory=False)
raw_data.head()

In [ ]:
# 
# TAKE A SUBSET OF THE ORIGINAL DATASET
# 
# This step selects a subset of the original dataset for training purposes. 
# It also assigns a record identifier field (ID_FIELD) for audit and reconciliation 
# purposes. 
# 
SELECTED_FIELDS = [
    'Survived',
    'Pclass',
    'Sex',
    'SibSp',
    'Parch',
    'Fare',
    'Embarked',
    'PassengerId',
    'Age',
]
ID_FIELD = 'PassengerId'
df = raw_data[SELECTED_FIELDS].dropna()
df[ID_FIELD] = df[ID_FIELD].astype('string')
df.head()

In [ ]:
# 
# ASSIGN VARIABLE TYPES
# 
# This step assigns variable types. OUTCOME_VAR identifies the dependent variable.
# Other arrays identify independent variables by type: categorical, numerical, or boolean. 
# 
OUTCOME_VAR = 'Age' 
CATEGORICAL_VARS = ['Sex', 'Embarked'] 
NUMERICAL_VARS = ['Pclass', 'SibSp', 'Parch', 'Fare'] 
BOOLEAN_VARS = ['Survived'] 

In [ ]:
# 
# START ANALYZR CLIENT
# 
# Access to the Analyzr API requires credentials. Contact your account manager or contact 
# our support team at https://support.analyzr.ai for more info. If you are a free tier user 
# you can skip this step altogether and use your local compute resources. 
# 
# For installation instructions on the Analyzr client see https://github.com/analyzr-ai/analyzr-sdk-python
# 
from analyzrclient import Analyzer
analyzer = Analyzer(host='analyzr4.api.g2m.ai')
analyzer.login()
analyzer.version()

In [ ]:
#
# TRAIN MODEL
#
# This step performs model training and cross-validation. If you do not have API access, e.g. free 
# tier users, you will need to replace the training command with your own model.fit() statement 
# for model training and cross-validation using local compute resources.  
# 
# Note also this version keeps polling the API while the model is training. For longer training runs 
# you may want to disable polling and manually check the status of your model (see next cell). 
# 
t_0 = datetime.datetime.now()
client_id = 'null'
algorithm = 'xgboost-regression'
verbose = True
res = analyzer.regression.train(
    df, client_id=client_id, 
    idx_var=ID_FIELD, outcome_var=OUTCOME_VAR, categorical_vars=CATEGORICAL_VARS, numerical_vars=NUMERICAL_VARS, 
    algorithm=algorithm, train_size=0.7, 
    buffer_batch_size=1000, verbose=verbose, poll=True, compressed=True, 
)
print('model_id: {}'.format(res['model_id']))
print('total time: {}'.format(datetime.datetime.now()-t_0))

In [ ]:
#
# CHECK STATUS OF MODEL RUN
# 
# For manual polling uncomment the lines below. 
# 
# res = analyzer.regression.check_status(model_id=res['model_id'], client_id=client_id, verbose=True) 
# res 

In [ ]:
#
# DISPLAY FEATURE IMPORTANCE 
#
sns.barplot(data=res['features'][0:15], x='Importance', y='Feature')

In [ ]:
#
# SHOW FEATURE IMPORTANCE TABLE
#
res['features']

In [ ]:
#
# SHOW ERROR STATISTICS
#
res['stats']